In [2]:
# Import Dependencies
import pandas as pd
import numpy as np

In [3]:
#File to load
schools_path = "raw_data/schools_complete.csv"

# Read the school csv and store into Pandas DataFrame
schools_df = pd.read_csv(schools_path, encoding="utf-8")
schools_df.head()

,School ID,name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [4]:
#File to load
students_path = "raw_data/students_complete.csv"

# Read the school csv and store into Pandas DataFrame
students_df = pd.read_csv(students_path, encoding="utf-8")
students_df.head()

,Student ID,name,gender,grade,school,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [26]:
# Calculate District Summary  
# Assumption passing score is >= 70 

totalSchools = len(schools_df["School ID"].unique())
totalStudents = schools_df["size"].sum()
totalBudget = schools_df["budget"].sum()

districtStudents_df = pd.merge(schools_df, students_df, how = 'left', left_on = 'name', right_on = 'school')

avgMathScore = districtStudents_df["math_score"].mean()
avgReadingScore = districtStudents_df["reading_score"].mean()
avgReadingScore

percentPassingMath_df = districtStudents_df.loc[districtStudents_df["math_score"] >= 70,:]

percentPassingMath = (percentPassingMath_df["Student ID"].count()/districtStudents_df["Student ID"].count())*100

percentPassingReading_df = districtStudents_df.loc[districtStudents_df["reading_score"] >= 70,:]

percentPassingReading = (percentPassingReading_df["Student ID"].count()/districtStudents_df["Student ID"].count())*100

overallPassingRate = (percentPassingReading+percentPassingMath)/2

district_summary = pd.DataFrame({"Total Schools":[totalSchools],
                             "Total Students":[totalStudents],
                             "Total Budget":[totalBudget],
                             "Average Math Score":[avgMathScore],
                             "Average Reading Score":[avgReadingScore],
                             "% Passing Math":[percentPassingMath],
                             "% Passing Reading":[percentPassingReading],
                             "Overall Passing Rate":[overallPassingRate]
                        
                                })
 
district_summary["Total Budget"] = district_summary["Total Budget"].map("${0:,.2f}".format)
    
district_summary = district_summary[["Total Schools", "Total Students", "Total Budget", "Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "Overall Passing Rate" ]].round(2)

district_summary



,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,15,39170,"$24,649,428.00",78.99,81.88,74.98,85.81,80.39


In [28]:
# add a column with a flag pass (1) or fail (2) if score is >= 70 and then sum

districtStudents_df["MathPass"] = np.where(districtStudents_df['math_score']>= 70,1,0)

districtStudents_df["ReadingPass"] = np.where(districtStudents_df['reading_score']>= 70,1,0)

grouped_districtstudents_df = districtStudents_df.groupby(["school"])

grouped_districtstudents_df.head()

# Calculate passing rate for math, reading and overall 

mathPassingRate = (grouped_districtstudents_df["MathPass"].sum()/grouped_districtstudents_df["size"].max())*100
readingPassingRate = (grouped_districtstudents_df["ReadingPass"].sum()/grouped_districtstudents_df["size"].max())*100
overallPassingRate = (mathPassingRate + readingPassingRate)/2


school_summary = pd.DataFrame({"School Type":grouped_districtstudents_df["type"].max(),
                               "Total Students":grouped_districtstudents_df["size"].max(),
                               "Total School Budget":grouped_districtstudents_df["budget"].max(),
                               "Per Student Budget":grouped_districtstudents_df["budget"].max()/grouped_districtstudents_df["size"].max(),
                               "Average Math Score":grouped_districtstudents_df["math_score"].mean(),
                               "Average Reading Score":grouped_districtstudents_df["reading_score"].mean(),
                               "% Passing Math":mathPassingRate,
                               "% Passing Reading":readingPassingRate,
                               "% Overall Passing Rate":overallPassingRate
                              })


school_summary["Total School Budget"] = school_summary["Total School Budget"].map("${0:,.2f}".format)
school_summary["Per Student Budget"] = school_summary["Per Student Budget"].map("${0:,.2f}".format)

school_summary = school_summary[["School Type", "Total Students", "Total School Budget", "Per Student Budget", "Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing Rate"]].round(2)

school_summary


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.05,81.03,66.68,81.93,74.31
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.06,83.98,94.13,97.04,95.59
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.71,81.16,65.99,80.74,73.36
Ford High School,District,2739,"$1,763,916.00",$644.00,77.10,80.75,68.31,79.30,73.80
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.35,83.82,93.39,97.14,95.27
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.29,80.93,66.75,80.86,73.81
Holden High School,Charter,427,"$248,087.00",$581.00,83.80,83.81,92.51,96.25,94.38
Huang High School,District,2917,"$1,910,635.00",$655.00,76.63,81.18,65.68,81.32,73.50
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.07,80.97,66.06,81.22,73.64


In [7]:
# Top 5 performing school by overall passing rate

school_summary.sort_values('% Overall Passing Rate', ascending=False).head(5)


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.06,83.98,94.13,97.04,95.59
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.42,83.85,93.27,97.31,95.29
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.35,83.82,93.39,97.14,95.27
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59,95.95,95.27
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.27,83.99,93.87,96.54,95.20


In [8]:
# Bottom 5 performing school by overall passing rate

school_summary.sort_values('% Overall Passing Rate', ascending=True).head(5)


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.84,80.74,66.37,80.22,73.29
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.71,81.16,65.99,80.74,73.36
Huang High School,District,2917,"$1,910,635.00",$655.00,76.63,81.18,65.68,81.32,73.50
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.07,80.97,66.06,81.22,73.64
Ford High School,District,2739,"$1,763,916.00",$644.00,77.10,80.75,68.31,79.30,73.80


In [9]:
#Math Scores by Grade

grouped_schoolGrade_df = districtStudents_df.groupby(["school", "grade"])

MathScoreByGrade_summary = pd.DataFrame({
                               "Average Math Score":grouped_schoolGrade_df["math_score"].mean(),
                              }).round(2)

MathScoreByGrade_summary.reset_index(inplace=True)

pivot_MathScorebyGrade = MathScoreByGrade_summary.pivot(index="school", columns="grade", values="Average Math Score")

pivot_MathScorebyGrade

grade,10th,11th,12th,9th
school,,,,
Bailey High School,77.00,77.52,76.49,77.08
Cabrera High School,83.15,82.77,83.28,83.09
Figueroa High School,76.54,76.88,77.15,76.40
Ford High School,77.67,76.92,76.18,77.36
Griffin High School,84.23,83.84,83.36,82.04
Hernandez High School,77.34,77.14,77.19,77.44
Holden High School,83.43,85.00,82.86,83.79
Huang High School,75.91,76.45,77.23,77.03
Johnson High School,76.69,77.49,76.86,77.19


In [24]:
#Reading Scores by Grade

grouped_schoolGrade_df = districtStudents_df.groupby(["school", "grade"])

ReadingScoreByGrade_summary = pd.DataFrame({
                               "Average Reading Score":grouped_schoolGrade_df["reading_score"].mean(),
                              }).round(2)

ReadingScoreByGrade_summary.reset_index(inplace=True)

pivot_ReadingScorebyGrade = ReadingScoreByGrade_summary.pivot(index="school", columns="grade", values="Average Reading Score")

pivot_ReadingScorebyGrade

grade,10th,11th,12th,9th
school,,,,
Bailey High School,80.91,80.95,80.91,81.30
Cabrera High School,84.25,83.79,84.29,83.68
Figueroa High School,81.41,80.64,81.38,81.20
Ford High School,81.26,80.40,80.66,80.63
Griffin High School,83.71,84.29,84.01,83.37
Hernandez High School,80.66,81.40,80.86,80.87
Holden High School,83.32,83.82,84.70,83.68
Huang High School,81.51,81.42,80.31,81.29
Johnson High School,80.77,80.62,81.23,81.26


In [12]:
# scores by school spending

districtStudents_df["Spending per Student"] = districtStudents_df["budget"]/districtStudents_df["size"]

# Create bins in which to place values based upon school size
bins = [0,585,615,645,675]
# Create labels for these bins
group_labels = ["<$585","$585-615","$615-645", "$645-675"]

districtStudents_df["Spending Ranges (Per Student)"] = pd.cut(districtStudents_df["Spending per Student"],bins,labels=group_labels)

districtStudents_df.head()

grouped_spendingRange_df = districtStudents_df.groupby(["Spending Ranges (Per Student)"])

grouped_spendingRange_df.head()


# Calculate passing rate for math, reading and overall 
mathPassingRate = (grouped_spendingRange_df["MathPass"].sum()/grouped_spendingRange_df["Student ID"].count())*100
readingPassingRate = (grouped_spendingRange_df["ReadingPass"].sum()/grouped_spendingRange_df["Student ID"].count())*100
overallPassingRate = (mathPassingRate + readingPassingRate)/2


spendingRange_summary = pd.DataFrame({
                               "Average Math Score":grouped_spendingRange_df["math_score"].mean(),
                               "Average Reading Score":grouped_spendingRange_df["reading_score"].mean(),
                               "% Passing Math":mathPassingRate,
                               "% Passing Reading":readingPassingRate,
                               "% Overall Passing Rate":overallPassingRate
                              })
spendingRange_summary = spendingRange_summary[["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing Rate"]].round(2)

spendingRange_summary





,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.36,83.96,93.70,96.69,95.19
$585-615,83.53,83.84,94.12,95.89,95.01
$615-645,78.06,81.43,71.40,83.61,77.51
$645-675,77.05,81.01,66.23,81.11,73.67


In [14]:
# scores by school size

# Create bins in which to place values based upon school size
bins = [0,1000,2000,5000]
# Create labels for these bins
group_labels = ["Small(<1000)","Medium(1000-2000)","Large(2000-5000)"]

districtStudents_df["School Size"] = pd.cut(districtStudents_df["size"],bins,labels=group_labels)

districtStudents_df.head()

grouped_schoolsize_df = districtStudents_df.groupby(["School Size"])

grouped_schoolsize_df.head()

mathPassingRate = (grouped_schoolsize_df["MathPass"].sum()/grouped_schoolsize_df["Student ID"].count())*100
readingPassingRate = (grouped_schoolsize_df["ReadingPass"].sum()/grouped_schoolsize_df["Student ID"].count())*100
overallPassingRate = (mathPassingRate + readingPassingRate)/2


schoolSizeRange_summary = pd.DataFrame({
                               "Average Math Score":grouped_schoolsize_df["math_score"].mean(),
                               "Average Reading Score":grouped_schoolsize_df["reading_score"].mean(),
                               "% Passing Math":mathPassingRate,
                               "% Passing Reading":readingPassingRate,
                               "% Overall Passing Rate":overallPassingRate
                              })
schoolSizeRange_summary = schoolSizeRange_summary[["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing Rate"]].round(2)

schoolSizeRange_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small(<1000),83.83,83.97,93.95,96.04,95.00
Medium(1000-2000),83.37,83.87,93.62,96.77,95.19
Large(2000-5000),77.48,81.20,68.65,82.13,75.39


In [18]:
# scores by school type

districtStudents_df.rename(columns={'type': 'School Type'}, inplace=True)

districtStudents_df.head()

grouped_schooltype_df = districtStudents_df.groupby(["School Type"])
grouped_schooltype_df.head()

mathPassingRate = (grouped_schooltype_df["MathPass"].sum()/grouped_schooltype_df["Student ID"].count())*100
readingPassingRate = (grouped_schooltype_df["ReadingPass"].sum()/grouped_schooltype_df["Student ID"].count())*100
overallPassingRate = (mathPassingRate + readingPassingRate)/2


schoolTypeRange_summary = pd.DataFrame({
                               "Average Math Score":grouped_schooltype_df["math_score"].mean(),
                               "Average Reading Score":grouped_schooltype_df["reading_score"].mean(),
                               "% Passing Math":mathPassingRate,
                               "% Passing Reading":readingPassingRate,
                               "% Overall Passing Rate":overallPassingRate
                              })
schoolTypeRange_summary = schoolTypeRange_summary[["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing Rate"]].round(2)

schoolTypeRange_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.41,83.90,93.70,96.65,95.17
District,76.99,80.96,66.52,80.91,73.71
